In [84]:
#import os
#import sys
#from pathlib import Path
#sys.path.append(str(Path(__file__).parent.parent))


In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from alns.rl.hybrid_agent import HybridAgent
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp

from experiment_storage.file_paths import FilePaths
from experiment_storage.storage import EvaluationStorage


# all_exp_ids = ['C101200_varinstsizeTSP', 'R101200_varinstsizeTSP', 'O101200_varinstsizeTSP',
#               'C101200_varinstsizeCVRP', 'R101200_varinstsizeCVRP', 'O101200_varinstsizeCVRP',
#               'C101200_varinstsizeVRPTW', 'R101200_varinstsizeVRPTW', 'O101200_varinstsizeVRPTW',
#               'C101200_varinstsizeHVRP', 'R101200_varinstsizeHVRP', 'O101200_varinstsizeHVRP',
#               'C101200_varinstsizeLRP', 'R101200_varinstsizeLRP', 'O101200_varinstsizeLRP']

# all_exp_ids = ['O101200_varinstsizeTSP',
#               'O101200_varinstsizeCVRP',
#               'O101200_varinstsizeVRPTW',
#               'O101200_varinstsizeHVRP',
#               'O101200_varinstsizeLRP']


# experiment result analysis 2023-7-24:
all_exp_ids = ['O101200_diffinitcwnoiseTSP', 'O101200_diffinitcwnoiseCVRP', 'O101200_diffinitcwnoiseHVRP', 'O101200_diffinitcwnoiseVRPTW', 'O101200_diffinitcwnoiseLRP',
               'O101200_diffinitrandTSP', 'O101200_diffinitrandCVRP', 'O101200_diffinitrandHVRP', 'O101200_diffinitrandVRPTW', 'O101200_diffinitrandLRP']


# parent_dir = "/Users/vdarvariu/experiment_data/ates"
#parent_dir = '/Users/shunee/github/ATES2022/experiment_data'
parent_dir = '/Users/shunee/github/ATES2022/all_variants_proportional_tours'


fp_out = FilePaths(parent_dir, "aggregate", setup_directories=True)

# filter_agent = [HybridAgent.algorithm_name]
filter_agent = None


# Final test set results

In [3]:
def compute_ci(data, confidence=0.95):
    a = np.array(data)
    n = len(a)
    se = sp.stats.sem(a)
    h = se * sp.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

def get_results_table(storage, exp_id):
    results = storage.get_evaluation_data(exp_id, task_type="alns")
    if filter_agent is not None:
        results = [r for r in results if r["algorithm"] in filter_agent]

    rows_df = pd.DataFrame(results)
    print(rows_df)
    pivot = pd.pivot_table(rows_df, values='obj_min', columns=["algorithm"], index=["cust_number"]).reset_index()
    # pivot["win_dqn_crw"] = pivot["dqnprob"] < pivot["classicrw"]
    # pivot.sort_index(axis=1, inplace=True)
    pivot = pd.DataFrame(pivot.to_records()).reset_index(drop=True)
    pivot = pivot[["cust_number", "randommdp", "classicrw", "dqn", "dqnprob", "hybrid", "vrwmdp"]]
    return pivot

In [4]:
for exp_id in all_exp_ids:
    print("=" * 20)
    print(exp_id)
    print("=" * 20)
    fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
    storage = EvaluationStorage(fp_in)
    # print(storage.retrieve_optimal_hyperparams(exp_id, False, task_type="tune", fp_in=fp_in))
    results_table = get_results_table(storage, exp_id)

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print(results_table)
    print()

O101200_diffinitcwnoiseTSP


KeyError: 'obj_min'

In [99]:
for exp_id in all_exp_ids:
    print("=" * 20)
    print(exp_id)
    print("=" * 20)
    fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
    storage = EvaluationStorage(fp_in)
    # print(storage.retrieve_optimal_hyperparams(exp_id, False, task_type="tune", fp_in=fp_in))
    results_table = get_results_table(storage, exp_id)

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print(results_table)
    print()

C101200_varinstsizeTSP
   cust_number   randommdp   classicrw         dqn     dqnprob      hybrid      vrwmdp
0           20  258.041645  258.442684  257.444458  245.685564  245.575343  243.189029
1           30  317.569630  317.461173  302.939433  283.396845  283.415783  281.088922
2           40  467.378461  468.450855  443.740142  404.248988  402.132615  395.985129
3           50  528.940827  524.278594  464.311450  430.778167  428.980487  423.130617

R101200_varinstsizeTSP
   cust_number   randommdp   classicrw         dqn     dqnprob      hybrid      vrwmdp
0           20  413.797585  413.142049  403.478867  391.980225  392.887455  391.590366
1           30  525.373547  524.894007  497.937425  481.686690  480.741590  477.880920
2           40  667.630894  670.060435  622.078157  599.857880  597.025006  594.887419
3           50  771.795954  765.061984  683.514780  660.261310  653.807131  654.229096

O101200_varinstsizeTSP
   cust_number   randommdp   classicrw         dqn     dqnp

In [89]:
all_dfs = []
for exp_id in all_exp_ids:
    fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
    storage = EvaluationStorage(fp_in)
    # print(storage.retrieve_optimal_hyperparams(exp_id, False, task_type="tune", fp_in=fp_in))
    exp_df = get_results_table(storage, exp_id)
    # print(storage.get_experiment_details(exp_id))
    details = storage.get_experiment_details(exp_id)
    exp_df['variant'] = details['problem_variant']
    exp_df['inst'] = exp_id[0]
    all_dfs.append(exp_df)

df = pd.concat(all_dfs)


In [90]:
def percentage_change(col1, col2):
    return ((col2 - col1) / col1) * 100


WR_dqnprob_dqn = (df["dqnprob"] < df["dqn"]).sum() / len(df)
print(f"Winning rate of DQN-prob over DQN: {WR_dqnprob_dqn:.3f}")

WR_hybrid_dqnprob = (df["hybrid"] < df["dqnprob"]).sum() / len(df)
print(f"Winning rate of Hybrid over DQN-prob: {WR_hybrid_dqnprob:.3f}")

WR_dqn_vrw = (df["dqn"] < df["vrwmdp"]).sum() / len(df)
print(f"Winning rate of DQN over VRW: {WR_dqn_vrw:.3f}")
WR_hybrid_vrw = (df["hybrid"] < df["vrwmdp"]).sum() / len(df)
print(f"Winning rate of Hybrid over VRW: {WR_hybrid_vrw:.3f}")

WR_crw_random = (df["classicrw"] < df["randommdp"]).sum() / len(df)
print(f"Winning rate of Classic RW over Random: {WR_hybrid_vrw:.3f}")


PC_rand_dqn = -percentage_change(df["randommdp"], df["dqn"]).mean()
print(f"Improvement in solution quality of DQN versus random operator selection: {PC_rand_dqn:.3f}%")

PC_rand_dqnprob = -percentage_change(df["randommdp"], df["dqnprob"]).mean()
print(f"Improvement in solution quality of DQN-prob versus random operator selection: {PC_rand_dqnprob:.3f}%")

PC_rand_vrw = -percentage_change(df["randommdp"], df["vrwmdp"]).mean()
print(f"Improvement in solution quality of VRW versus random operator selection: {PC_rand_vrw:.3f}%")

PC_rand_hybrid = -percentage_change(df["randommdp"], df["hybrid"]).mean()
print(f"Improvement in solution quality of Hybrid versus random operator selection: {PC_rand_hybrid:.3f}%")

PC_rand_crw = -percentage_change(df["randommdp"], df["classicrw"]).mean()
print(f"Improvement in solution quality of CRW versus random operator selection: {PC_rand_crw:.3f}%")

# PC_hybrid_vrw = -percentage_change(df["vrwmdp"], df["hybrid"]).mean()
# print(f"Improvement in solution quality of Hybrid versus VRW: {PC_hybrid_vrw:.3f}%")

Winning rate of DQN-prob over DQN: 0.967
Winning rate of Hybrid over DQN-prob: 0.867
Winning rate of DQN over VRW: 0.167
Winning rate of Hybrid over VRW: 0.633
Winning rate of Classic RW over Random: 0.633
Improvement in solution quality of DQN versus random operator selection: 10.388%
Improvement in solution quality of DQN-prob versus random operator selection: 14.581%
Improvement in solution quality of VRW versus random operator selection: 14.691%
Improvement in solution quality of Hybrid versus random operator selection: 15.724%
Improvement in solution quality of CRW versus random operator selection: 0.157%


In [91]:
WR_hybrid_dqnprob

0.8666666666666667